# DSE HA-3
# **Creating your own Sorting Hogwarts Hat ✨**

The aim of this task is to constuct ML model that predicts  Hogwarts Houses based on students forces (the target variable is `Hogwarts House`)

Good luck, dear wizards!

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("https://raw.githubusercontent.com/Shuaynat/DSE-23-24/main/02-home-assignments/ha-03-practice/ha-3_dataset.csv")


In [ ]:
data.head()

,Index,Hogwarts House,First Name,Last Name,Birthday,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
0,0,Ravenclaw,Tamara,Hsu,2000-03-30,Left,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89
1,1,Slytherin,Erich,Paredes,1999-10-14,Right,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45
2,2,Ravenclaw,Stephany,Braun,1999-11-03,Left,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42
3,3,Gryffindor,Vesta,Mcmichael,2000-08-19,Left,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64
4,4,Gryffindor,Gaston,Gibbs,1998-09-27,Left,60158.0,436.775204,-7.820623,NaN,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-0.264070,-256.38730,157.98


## 1 Data preparation

Follow several preparation steps:

1. Drop columns that we will not use for prediction: last name, first name and index.
2. Have a look at the correlation plot. If you see that some features are too correlated - drop some of them
3. Fill mising features with mean taret encoding
4. Convert categorical variables `Best Hand` and `Hogwart House` to numerical.
5. Create from `Birthday` new fearures `day`, `month`, `year`
6. Standardize features
7. Separate data into test and train sets (leave 33% of data in test set)


In [ ]:
# drop the columns
df = data.drop(columns=['First Name', 'Last Name', 'Index'])


In [ ]:
# plot the correlation matrix
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')


<ipython-input-21-c115565a62ce>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
Arithmancy,1.000000,-0.033355,-0.018613,0.037280,-0.003485,-0.055085,0.076992,-0.069778,-0.004673,-0.302859,-0.042129,-0.100896,-0.166747
Astronomy,-0.033355,1.000000,0.020210,-1.000000,0.481663,-0.578511,-0.192767,-0.409449,-0.440900,-0.558429,-0.012876,-0.521203,0.523958
Herbology,-0.018613,0.020210,1.000000,-0.023810,0.549578,0.437113,-0.042959,0.510289,0.537615,-0.106415,0.047737,0.746614,-0.271344
Defense Against the Dark Arts,0.037280,-1.000000,-0.023810,1.000000,-0.479310,0.576553,0.193138,0.409559,0.442231,0.556683,0.010865,0.521115,-0.526304
Divination,-0.003485,0.481663,0.549578,-0.479310,1.000000,0.249272,0.435663,-0.170008,-0.166173,-0.482459,0.023438,0.395282,0.402269
Muggle Studies,-0.055085,-0.578511,0.437113,0.576553,0.249272,1.000000,0.569814,0.289745,0.266522,0.184076,0.050356,0.847607,-0.184862
Ancient Runes,0.076992,-0.192767,-0.042959,0.193138,0.435663,0.569814,1.000000,-0.472979,-0.449731,-0.122483,-0.010400,0.354147,0.559944
History of Magic,-0.069778,-0.409449,0.510289,0.409559,-0.170008,0.289745,-0.472979,1.000000,0.849203,0.602211,0.029390,0.539685,-0.896283
Transfiguration,-0.004673,-0.440900,0.537615,0.442231,-0.166173,0.266522,-0.449731,0.849203,1.000000,0.568442,0.044615,0.549467,-0.873673
Potions,-0.302859,-0.558429,-0.106415,0.556683,-0.482459,0.184076,-0.122483,0.602211,0.568442,1.000000,0.003523,0.248049,-0.558594


Comments on correlation matrix:
- Charms and Herbology are high correlated - 0.74661
- Charms and Muggle strudies are highly correlated - 0.847
- Flying is highly correlated to History of Magic - 0.896
- Flying is highly correlated to Transfiguration - 0.874
- History of Magic and Transiguration are gighly correlated - 0.849

In [ ]:
# drop the features that are higly correlated
df = df.drop(columns=['Charms', 'History of Magic', 'Transfiguration', 'Flying'])

In [ ]:
# Fill mising features with mean taret encoding

na_means = {
    "Arithmancy": df['Arithmancy'].mean(),
    "Astronomy": df['Astronomy'].mean(),
    "Herbology": df['Herbology'].mean(),
    "Defense Against the Dark Arts": df['Defense Against the Dark Arts'].mean(),
    "Divination": df['Divination'].mean(),
    "Muggle Studies": df['Muggle Studies'].mean(),
    "Ancient Runes": df['Ancient Runes'].mean(),
    "Potions": df['Potions'].mean(),
    "Care of Magical Creatures": df['Care of Magical Creatures'].mean()
}

df.fillna(na_means, inplace=True)


In [ ]:
# Convert categorical variables Best Hand and Hogwart House to numerical.

# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder

# Creating a instance of label Encoder.
le = LabelEncoder()

# Using .fit_transform function to fit label
# encoder and return encoded label
df['Best Hand'] = le.fit_transform(df['Best Hand'])
df['Hogwarts House'] = le.fit_transform(df['Hogwarts House'])


In [ ]:
# Create from Birthday new fearures day, month, year

df["Birthday"] = df["Birthday"].astype("datetime64[ns]")

df["year"] = df["Birthday"].dt.year
df["month"] = df["Birthday"].dt.month
df["day"] = df["Birthday"].dt.day

df = df.drop(columns=['Birthday'])

In [ ]:
# Standardize features

from sklearn.preprocessing import StandardScaler

object= StandardScaler()
df_new = object.fit_transform(df)

In [ ]:
# Separate data into test and train sets (leave 33% of data in test set)

# import modules
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# get the locations
X = df.drop(columns=['Hogwarts House'])
y = df['Hogwarts House']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.33, random_state=42)


## 2. Multiclass logit model with regularization


Estimate multiclass logit model:

1. Use sklearn realization of logistic regression in [SGDC classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)


2. Use cross-validation to choose type of regularization (penalty equal to `l1`, `l2`, or `elastic net`) and regularization weight (`alpha` in  `range(start=0.1, 1, step=0.1)`)


In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import RepeatedKFold


# Define the range of alphas to search
alphas = [0.1 * i for i in range(1, 11)]

# Set up cross-validation strategy
cv_strategy = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# Set up models for different types of regularization
lasso = LassoCV(alphas=alphas, cv=cv_strategy)
ridge = RidgeCV(alphas=alphas, cv=cv_strategy)
elastic_net = ElasticNetCV(alphas=alphas, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 1], cv=cv_strategy)

# Fit models and find best alpha
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic_net.fit(X_train, y_train)

# Compare models and select the one with the best cross-validation score
models = {
    'Lasso (L1)': (lasso, lasso.alpha_),
    'Ridge (L2)': (ridge, ridge.alpha_),
    'ElasticNet': (elastic_net, elastic_net.alpha_)
}

best_score = -1
best_model = None
best_alpha = None
best_name = None

for name, (model, alpha) in models.items():
    score = model.score(X, y)  # Assuming R^2 score or replace with model-specific score
    if score > best_score:
        best_score = score
        best_model = model
        best_alpha = alpha
        best_name = name

# Display the best model type and regularization weight
print(f"Best model: {best_name} with alpha = {best_alpha}")

Best model: Ridge (L2) with alpha = 1.0


## 3. Calculate average marginal effects




For each of the variables calculate average marginal effect on the probability.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

sgd_clf_best = SGDClassifier(loss='log', penalty='l2', alpha=0.1, random_state=1)
sgd_clf_best.fit(X_train, y_train)

results = permutation_importance(sgd_clf_best, X_train, y_train, n_repeats=30, random_state=1)

average_marginal_effects = results.importances_mean

ame_dict = dict(zip(X.columns, average_marginal_effects))

ame_dict

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


{'Best Hand': 0.0,
 'Arithmancy': -0.01421019900497512,
 'Astronomy': 0.1909514925373134,
 'Herbology': 0.0,
 'Defense Against the Dark Arts': 0.0,
 'Divination': 0.0,
 'Muggle Studies': 0.2536380597014925,
 'Ancient Runes': 0.02978855721393036,
 'Potions': 0.0,
 'Care of Magical Creatures': 0.0,
 'year': 0.0,
 'month': 0.0,
 'day': 0.0}

## 4. Build logit with PCA transformation


Extract 5 principal components and evaluate logit model using them as explanatory variables

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# PCA to reduce the data to 5 principal components
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33, random_state=42)

# Train a logistic regression model using the principal components
logit_model = LogisticRegression()
logit_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logit_model.predict(X_test)

# Evaluate the logistic model
print("Classification report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# To see how PCA features contribute to the logit model, you can look at the coefficients
print("Logit model coefficients for principal components:")
print(logit_model.coef_)

Classification report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97        98
           1       0.97      0.99      0.98       186
           2       0.99      0.95      0.97       153
           3       0.94      0.97      0.95        91

    accuracy                           0.97       528
   macro avg       0.97      0.97      0.97       528
weighted avg       0.97      0.97      0.97       528

Accuracy Score: 0.9696969696969697
Logit model coefficients for principal components:
[[ 1.22501068e-05 -1.82535632e-03 -6.03530457e-03  2.40754572e-02
   1.15028500e-04]
 [ 1.73157610e-05 -5.68502261e-03  1.40163433e-03 -2.26617435e-02
   1.03220981e-04]
 [-4.95294280e-06  5.04650455e-03 -1.26960676e-03  3.39302773e-03
  -2.59038087e-04]
 [-2.46129250e-05  2.46387437e-03  5.90327701e-03 -4.80674149e-03
   4.07886055e-05]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 5. Build classification forest


You are free to add PCA-transformed explanatory variables as predictors. Tune parameters by cross-validation on the grid of your choice

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pca = PCA(n_components=5)
rf_clf = RandomForestClassifier(random_state=42)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('rf', rf_clf),
])
param_grid = {
    'pca__n_components': [5],
    'rf__n_estimators': [50, 100, 150],
    'rf__max_depth': [None, 5, 10, 15, 20],
}
scorer = 'accuracy'
grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
y_pred = grid_search.best_estimator_.predict(X_test)
print(f"Accuracy:", accuracy_score(y_test, y_pred))

Best Parameters: {'pca__n_components': 5, 'rf__max_depth': 10, 'rf__n_estimators': 150}
Accuracy: 0.9696969696969697


## 6. Model Race: evaluate models (Logit, Logit + PCA, Classification forest) on the test dataset and choose the best one

In [ ]:
# Logistic Regression
logit_model = LogisticRegression(random_state=10, max_iter=10000)
logit_model.fit(X_train, y_train)
y_pred_logit = logit_model.predict(X_test)
accuracy_logit = accuracy_score(y_test, y_pred_logit)

print("Logistic Regression Accuracy:", accuracy_logit)

Logistic Regression Accuracy: 0.9791666666666666


   



---

In [ ]:
# Logistic Regression with PCA
pca = PCA(n_components=5)
logit_pca_model = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', pca),
    ('logit', LogisticRegression(random_state=10, max_iter=1000)),
])
logit_pca_model.fit(X_train, y_train)
y_pred_logit_pca = logit_pca_model.predict(X_test)
accuracy_logit_pca = accuracy_score(y_test, y_pred_logit_pca)

print("Logistic Regression with PCA Accuracy:", accuracy_logit_pca)

Logistic Regression with PCA Accuracy: 0.9753787878787878


In [ ]:
# Random Forest
rf_model = RandomForestClassifier(random_state=10, n_estimators=100, max_depth=None)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("Random Forest Accuracy:", accuracy_rf)

Random Forest Accuracy: 0.9753787878787878


![Alt Text](https://gifdb.com/images/high/gryffindor-sorting-hat-owbd80ftf54oai7i.webp)


